# Statistical Indocators from Anomaly Hotspot of Agricultural Production (ASAP)

ASAP is a platform used for early warning of  upcoming weather anomaly that leads to fluctuation of agricultural production.

- Genral link: https://mars.jrc.ec.europa.eu/asap/
- Download link: https://mars.jrc.ec.europa.eu/asap/download.php
- Senegal country assesment: https://mars.jrc.ec.europa.eu/asap/country.php?cntry=217
- GDrive data: https://drive.google.com/file/d/1JrPN5f_o5vTgIkLJLs0n04rjc49RsDPX/view?usp=sharing

In [1]:
# imports
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

plt.style.use('classic')
%matplotlib inline

Get data_path folder. Under subfolder region, we have statistics in gaul1 resolution.

In [2]:
data_folder = Path.cwd().parent.parent.joinpath('data', 'external', 'asap', 'region')


In [3]:
df_asap = pd.read_csv(str(Path(data_folder).joinpath('SEN_asap_region.csv')))

In [112]:
df_asap['region_name'].unique()

array(['Kolda', 'Fatick', 'Dakar', 'Kedougou', 'Tambacounda', 'Sedhiou',
       'Kaffrine', 'Kaolack', 'Thies', 'Ziguinchor', 'Diourbel',
       'Saint louis', 'Louga', 'Matam'], dtype=object)

In [82]:
import plotly.graph_objects as go
import plotly.express as px


In [ ]:

df_temp = df_asap #[(df_asap['unit_name'] == 'Dakar') | (df_asap['unit_name'] == 'Velingara')]
g = sns.FacetGrid(data = df_temp,
                  col = 'variable_name', row = 'region_name',sharey=False,
                  height = 5)
g.map(sns.lineplot, 'dec_day', 'value')
df_curr = df_temp[df_temp['year'] == 2020]
axes = g.axes.flatten()

for ax in axes:
    title = str(ax.title)
    unit = title.split("= ")[1].split()[0]
    var = title.split("= ")[2].split("'")[0]
    ax.plot(df_curr.loc[(df_curr['region_name'] == unit) &
                             (df_curr['variable_name'] == var), 'dec_day'],
                     df_curr.loc[(df_curr['region_name'] == unit) &
                             (df_curr['variable_name'] == var), 'value'], '*', color = 'r')
    ax.legend(['average', 'last year'])

In [5]:
df_asap.head(1)

,region_name,variable_name,sensor_name,class_name,date,value,year,month,day,yearday,dec_day,season_year,season_dec_day
0,Kolda,Temperature,ECMWF,Rangeland during growing cycle,20120101,26.903,2012,1,1,1,0,2011.0,260.0


In [105]:
def plot_region_weather(reg="Kolda", weather="Rainfall", year=2019):
    df_temp = df_asap[["region_name", "variable_name", "dec_day", "value"]].copy()
    df_temp = df_temp[df_temp["variable_name"] == weather]

    df_m = df_temp.groupby(['region_name', 'dec_day']).mean()
    df_se = df_temp.groupby(['region_name', 'dec_day']).sem()

    fig = go.Figure()
    m = df_m.loc[reg]
    s = df_se.loc[reg]
    x = m.index.values
    m = m.values.flatten()
    s = s.values.flatten()
    x = np.hstack([x, x[::-1]])
    y_up = m+s*1.96
    y_low = m-s*1.96
    y_low = y_low[::-1]

    fig.add_trace(go.Scatter(
        x=x,
        y=np.hstack([y_up, y_low]),
        fill='toself',
        fillcolor='rgba(0,100,80,0.2)',
        line_color='rgba(255,255,255,0)',
        showlegend=True,
        name='confidence interval',
    ))
    curr = df_asap[(df_asap["year"] == year) & (df_asap["variable_name"] == weather) &
    (df_asap["region_name"] == reg)].copy()
    curr.sort_values(by="dec_day")
    curr = curr[["dec_day", "value"]]
    curr = curr.groupby("dec_day").mean()
    # x = curr["dec_day"].values.flatten()
    # y = curr["value"].values.flatten()
    fig.add_trace(go.Scatter(x=curr.index, y=curr.values.flatten(), name="Last year"))

    fig.update_traces(mode='lines')
    fig.show()

In [111]:
plot_region_weather(weather="NDVI")

In [41]:
df_asap[(df_asap['region_name'] == "Kolda") & (df_asap['variable_name'] == "")]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         4.53053435e-02,  1.08678916e-01,  1.12346342e+00,
         2.59119265e+00,  8.05091488e+00,  1.29298136e+01,
         3.20071195e+01,  4.72809509e+01,  6.00240553e+01,
         5.88695693e+01,  7.91184615e+01,  1.05844647e+02,
         1.05171704e+02,  1.24267128e+02,  1.64992615e+02,
         1.36068615e+02,  1.14210680e+02,  1.04688928e+02,
         5.27111110e+01,  4.41015431e+01,  2.01923042e+01,
         2.33480389e+00,  3.25181980e-01,  3.15976147e-02,
         3.76655836e-01,  2.02569970e-01,  1.74613677e-01],
       [ 2.43863227e-02,  5.51175305e-02,  3.16469164e-01,
        -1.04865036e-02,  2.34846864e-02,  1.18380722e+00,
         2.27647360e+00,  6.24629028e+00,  1.68580557e+01,
         4.66109049e+01,  5.45709862e+01,  6.34058852e+01,
         8.33964408e+01,  5.65589827e+01,  3.71292956e+

In [110]:
df_asap['variable_name'].unique()



array(['Temperature', 'NDVI Cumulated - zscore', 'SPI - 3 months',
       'Water Satisfaction Index (WSI)', 'NDVI - zscore', 'NDVI',
       'Rainfall'], dtype=object)

In [ ]:
df_temp = df_asap[['region_name', 'variable_name', 'year', 'dec_day', 'value']]

In [ ]:
df_temp.head()

In [ ]:
df_temp2 = df_temp[(df_temp['region_name'] == 'Dakar')& (df_temp['variable_name'] == 'Rainfall')]

g = sns.FacetGrid(data = df_temp2[['year', 'dec_day', 'value']], row='year')
g.map(sns.lineplot, 'dec_day', 'value')

In [ ]:

# Generate a set of sample data
np.random.seed(0)
indices = pd.MultiIndex.from_product((range(5), range(5), range(5)),
 names=('label0', 'label1', 'label2'))
data = pd.DataFrame(np.random.uniform(0, 100, size=len(indices)),
 index=indices, columns=('value',)).reset_index()

def draw_heatmap(*args, **kwargs):
    data = kwargs.pop('data')
    d = data.pivot(index=args[1], columns=args[0], values=args[2])
    sns.heatmap(d, **kwargs)

fg = sns.FacetGrid(data, col='label0')
fg.map_dataframe(draw_heatmap, 'label1', 'label2', 'value', cbar=False, square = True)
# get figure background color
facecolor=plt.gcf().get_facecolor()

In [108]:
df_asap[df_asap['region_name'] == "Kolda"]


,region_name,variable_name,sensor_name,class_name,date,value,year,month,day,yearday,dec_day,season_year,season_dec_day
0,Kolda,Temperature,ECMWF,Rangeland during growing cycle,20120101,26.903,2012,1,1,1,0,2011.0,260.0
1,Kolda,Temperature,ECMWF,Rangeland during growing cycle,20120111,27.939,2012,1,11,11,10,2011.0,270.0
2,Kolda,Temperature,ECMWF,Rangeland during growing cycle,20120501,33.000,2012,5,1,122,120,2012.0,20.0
3,Kolda,Temperature,ECMWF,Rangeland during growing cycle,20120511,31.339,2012,5,11,132,130,2012.0,30.0
4,Kolda,Temperature,ECMWF,Rangeland during growing cycle,20120521,31.639,2012,5,21,142,140,2012.0,40.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
85473,Kolda,Rainfall,ECMWF-CHIRPS,Crop during growing cycle,20201011,30.356,2020,10,11,285,280,2020.0,180.0
85474,Kolda,Rainfall,ECMWF-CHIRPS,Crop during growing cycle,20201021,2.075,2020,10,21,295,290,2020.0,190.0
85475,Kolda,Rainfall,ECMWF-CHIRPS,Crop during growing cycle,20201101,1.102,2020,11,1,306,300,2020.0,200.0
85476,Kolda,Rainfall,ECMWF-CHIRPS,Crop during growing cycle,20201111,0.136,2020,11,11,316,310,2020.0,210.0
